In [1]:
import cv2
from PIL import Image
import pytesseract
import warnings
warnings.filterwarnings('ignore')
from Preprocessing_cheatsheet import say_hello, show_image, start, grayscale, noise_removal, thin_font, thick_font, deskew_image, remove_border,Blur
import numpy as np
import os

In [2]:
os.chdir("D:\Internship\Studjee\Dataset")
os.getcwd()

'D:\\Internship\\Studjee\\Dataset'

In [3]:
for image_file in os.listdir():
    print(image_file)

2022_01_11_12_38_PM_Office_Lens_4.jpg
2022_01_11_12_38_PM_Office_Lens_4_contours.jpg
compress_16283973586881672795621.jpeg
compress_16283973586881672795621_contours.jpeg
compress_1629446272360525281284.jpeg
compress_1629446272360525281284_contours.jpeg
compress_IMG20220103105239.jpeg
compress_IMG20220103105239_contours.jpeg
compress_IMG_20210511_130407.jpeg
compress_IMG_20210511_130407_contours.jpeg
compress_IMG_20210808_125203.jpeg
compress_IMG_20210808_125203_contours.jpeg
compress_IMG_20220104_112022.jpeg
compress_IMG_20220104_112022_contours.jpeg
compress_IMG_20220104_112032.jpeg
compress_IMG_20220104_112032_contours.jpeg
philosophy-1.jpg
philosophy-1_contours.jpg


In [11]:
def get_bounding_box(image_file):
    image = cv2.imread(image_file)

    # Convert the image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the blue color in HSV
    lower_blue = np.array([50, 5, 15])
    upper_blue = np.array([500, 455, 455])

    # Create a mask using the inRange function
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Apply the mask to the original image
    result = cv2.bitwise_and(image, image, mask=blue_mask)
    # Display the original image with contours
    cv2.imshow('Blue Filter', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    # Convert the filtered image to grayscale
    gray_filtered = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    # Find contours in the filtered image
    contours, _ = cv2.findContours(gray_filtered, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    contour_image = image.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    # Display the original image with contours
    cv2.imshow('Original Image with Contours', contour_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    if contours:
        # Create a blank image with the same size as the original image
        contour_image = np.zeros_like(contour_image)

        # Draw contours on the blank image
        cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

        # Display the image with only the contours
        cv2.imshow('Image with Contours Only', contour_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        gray_contour_image = grayscale(contour_image)
        cv2.imwrite("images/gray_contour_image.jpg",gray_contour_image)
        cv2.imshow('gray_contour_image', gray_contour_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()        

        blur_image = Blur(gray_contour_image)
        thresh_value,thresh = cv2.threshold(blur_image,0,150,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        # print(thresh)
        cv2.imwrite("images\contour_thresh.jpg",thresh)
        cv2.imshow('contour_thresh', thresh)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 

        blank_image = np.zeros_like(image)

        # Draw contours on the blank image
        cv2.drawContours(blank_image, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

        ret,thresh2 = cv2.threshold(thresh,127,255,cv2.THRESH_BINARY_INV)
        cv2.imwrite("images\contour_thresh_inv.jpg",thresh2)
        cv2.imshow('contour_thresh_inv', thresh2)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 


        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))

        dilate = cv2.dilate(thresh,kernel,iterations=5)
        cv2.imwrite("images\contour_dialate.jpg",dilate)
        cv2.imshow('contour_dialate', dilate)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 
    
    
    if contours:
        contours, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contour_only_image = cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
        location,extension = image_file.split('.')
        new_path = location+"_contours."+extension
        print(new_path)
        
        cv2.imwrite(new_path,contour_only_image)
        cv2.imshow('contour_only_image', contour_only_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 

    else:
        print("No contours found.")


In [ ]:
# get_bounding_box("D:\Internship\Studjee\Dataset\compress_IMG20220103105239.jpeg")
for image_file in os.listdir():
    get_bounding_box(image_file)

2022_01_11_12_38_PM_Office_Lens_4_contours.jpg
2022_01_11_12_38_PM_Office_Lens_4_contours_contours.jpg
2022_01_11_12_38_PM_Office_Lens_4_contours_contours_contours.jpg
compress_16283973586881672795621_contours.jpeg
compress_16283973586881672795621_contours_contours.jpeg
compress_16283973586881672795621_contours_contours_contours.jpeg
compress_1629446272360525281284_contours.jpeg


In [6]:
import cv2
import numpy as np
import pytesseract

# Load the image
image = cv2.imread("D:\Internship\Studjee\Dataset\compress_IMG20220103105239.jpeg")
# image = cv2.imread("D:\Internship\Studjee\Dataset\compress_IimaMG_20220104_112022.jpeg")
# image = cv2.imread("D:\Internship\Studjee\Dataset\philosophy-1.jpg")
# image = cv2.imread("D:\Internship\Studjee\Dataset\2022_01_11_12_38_PM_Office_Lens_4.jpg")



# Convert the image to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the lower and upper bounds for the blue color in HSV
lower_blue = np.array([50, 5, 15])
upper_blue = np.array([500, 455, 455])

# Create a mask using the inRange function
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Apply the mask to the original image
result = cv2.bitwise_and(image, image, mask=blue_mask)
# Display the original image with contours
cv2.imshow('Blue Filter', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Convert the filtered image to grayscale
gray_filtered = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# Find contours in the filtered image
contours, _ = cv2.findContours(gray_filtered, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
contour_image = image.copy()
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

# Display the original image with contours
cv2.imshow('Original Image with Contours', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [9]:
if contours:
    # Create a blank image with the same size as the original image
    contour_image = np.zeros_like(contour_image)

    # Draw contours on the blank image
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    # Display the image with only the contours
    cv2.imshow('Image with Contours Only', contour_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    gray_contour_image = grayscale(contour_image)
    cv2.imwrite("images/gray_contour_image.jpg",gray_contour_image)
    show_image("images/gray_contour_image.jpg")

    blur_image = Blur(gray_contour_image)
    thresh_value,thresh = cv2.threshold(blur_image,0,150,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # print(thresh)
    cv2.imwrite("images\contour_thresh.jpg",thresh)
    show_image("images\contour_thresh.jpg")
    
    blank_image = np.zeros_like(image)

    # Draw contours on the blank image
    cv2.drawContours(blank_image, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    
    ret,thresh2 = cv2.threshold(thresh,127,255,cv2.THRESH_BINARY_INV)
    cv2.imwrite("images\contour_thresh_inv.jpg",thresh2)
    show_image("images\contour_thresh_inv.jpg")
    
    
#     blur_image = Blur(thresh2,(3,3),0)   
#     cv2.imwrite("images\thresh2_blur.jpg",blur_image)
#     cv2.imshow("","images\thresh2_blur.jpg")
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()  
#     show_image("images\thresh2_blur.jcontourpg")

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    
#     erosion = cv2.erode(thresh,kernel,iterations=3)
#     cv2.imwrite("images\contour_erode.jpg",dilate)
#     show_image("images\contour_erode.jpg")

    dilate = cv2.dilate(thresh,kernel,iterations=5)
    cv2.imwrite("images\contour_dialate.jpg",dilate)
    show_image("images\contour_dialate.jpg")

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [10]:
 if contours:
    contours, _ = cv2.findContours(dialate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_only_image = cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
    cv2.imwrite("images/contour_only_image.jpg",contour_only_image)
    show_image("images/contour_only_image.jpg")
    
    # Display the image with only the contours
    
else:
    print("No contours found.")

NameError: name 'dialate' is not defined